<a href="https://colab.research.google.com/github/VishakBharadwaj94/CNN_starter/blob/main/MNIST_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import torchvision


train_data = torchvision.datasets.MNIST('./',download=True)
train_data.data.shape

torch.Size([60000, 28, 28])

In [91]:
ints = torch.randint(0,9,size=(60_000,))

In [92]:
ints[:10]

tensor([6, 1, 0, 5, 8, 5, 2, 8, 4, 3])

In [93]:
import torch 
from torch import nn 
from torch.nn import functional as F

class Dataset:

  def __init__(self):
    self.data = train_data.data.float()
    self.targets = train_data.targets
    self.ints = ints
  
  def __getitem__(self,idx):
    image = torch.unsqueeze(self.data[idx],0)
    int_no = ints[idx]
    target = self.targets[idx]

    return image,int_no,target,int_no+target
    
  def __len__(self):
    
    return len(self.targets)

    

In [94]:
train_data = Dataset()

In [95]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

        #addition
        self.linear1 = nn.Linear(2,20)
        self.linear2 = nn.Linear(20,1)

    def forward(self, x,x2):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        out1 = self.fc2(x)

        #addition
        x1 = torch.argmax(F.softmax(out1),1)
        inp = torch.cat((x1[:,None],x2[:,None]),dim=1).float()
        out = self.linear1(inp)
        out = F.relu(out)
        out = self.linear2(out)


        return F.log_softmax(out1),out

In [96]:
network = Net().cuda()
optimizer = torch.optim.AdamW(network.parameters(), lr=0.1)

In [97]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data,shuffle=True,batch_size=256)

In [102]:
def train(epoch):

  network.train()
  losses = 0.
  for data,int_no,target,sum_int in train_loader:
    optimizer.zero_grad()
    output1,output2 = network(data.float().cuda(),int_no.float().cuda())
    loss1 = F.nll_loss(output1, target.cuda())
    loss2 = F.mse_loss(output2,sum_int.float().cuda())
    loss = (loss1+loss2)/2
    loss.backward()
    optimizer.step()
    losses+=loss
  print(f"epoch {epoch+1},Loss {losses}")

In [103]:
epochs = 10

for epoch in range(epochs):
  train(epoch)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Using a target size (torch.Size([96])) that is different to the input size (torch.Size([96, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if __name__ == '__main__':


epoch 1,Loss 2038.466064453125
epoch 2,Loss 2041.0753173828125
epoch 3,Loss 2073.2119140625
epoch 4,Loss 2039.6890869140625
epoch 5,Loss 2052.438232421875
epoch 6,Loss 2045.7342529296875
epoch 7,Loss 2061.76171875
epoch 8,Loss 2043.4609375
epoch 9,Loss 2047.882568359375
epoch 10,Loss 2042.280029296875


# IGNORE STUFF BELOW THIS LINE

In [26]:
add_sum = add1 + add2

In [5]:
from torch import nn
from torch.nn import functional as F 

class Net(nn.Module):

  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(2,20)
    self.linear2 = nn.Linear(20,1)

  def forward(self,x1,x2):
    inp = torch.cat((x1[None],x2[None])).float()
    out = self.linear1(inp)
    out = F.relu(out)
    out = self.linear2(out)

    return out 

In [34]:
net = Net()
optim = torch.optim.AdamW(net.parameters(),lr=0.1)
criterion = nn.MSELoss()

In [35]:
for i in range(len(add1)):

  out = net(add1[i],add2[i])
  loss = criterion(out,add_sum[i].float())
  optim.zero_grad()
  loss.backward()
  optim.step()

  if i%500==0: print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(9.5540, grad_fn=<MseLossBackward>)
tensor(0.0009, grad_fn=<MseLossBackward>)
tensor(0.0118, grad_fn=<MseLossBackward>)
tensor(0.0004, grad_fn=<MseLossBackward>)
tensor(0.3194, grad_fn=<MseLossBackward>)
tensor(0.0007, grad_fn=<MseLossBackward>)
tensor(0.0010, grad_fn=<MseLossBackward>)
tensor(3.0294, grad_fn=<MseLossBackward>)
tensor(0.0036, grad_fn=<MseLossBackward>)
tensor(0.0049, grad_fn=<MseLossBackward>)
tensor(0.0087, grad_fn=<MseLossBackward>)
tensor(0.0001, grad_fn=<MseLossBackward>)


In [44]:
x1 = torch.tensor(5)
x2 = torch.tensor(12)

In [1]:
import torch 

add1= torch.randint(0,9,size=[6000])
add2= torch.randint(0,9,size=[6000])
torch.cat((add1[0][None],add2[0][None])).shape

torch.Size([2])

In [45]:
net(x1,x2)

tensor([17.0277], grad_fn=<AddBackward0>)